In [1]:
import os
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers, callbacks, regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [2]:
seed_value = 42

os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [3]:
model_version = 'v5.0'

In [4]:
with open('data_schema.json', 'r') as f:
    config = json.load(f)
    
spectro_cols = config['spectro_cols']
photo_cols = config['photometry_cols']
generic_cols = config['generic_cols']
target_col = config['target_col']

In [5]:
df = pd.read_csv(f"data/processed/spectrometry_photometry.csv")

X = df[spectro_cols + photo_cols + generic_cols]
y = df[target_col]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed_value)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# First Model: Spectroscopic Features + Photometric Features
full_input = keras.Input(shape=(X_train_scaled.shape[1]), name = 'full_input')
x = layers.Dense(128, activation='relu', name='Dense_128')(full_input)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu', name='Dense_256')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu', name='Dense_64')(x)
x = layers.Dropout(0.2)(x)
output = layers.Dense(1, name='logM_star')(x)

full_model = keras.Model(inputs=full_input, outputs=output, name='Full_Model')
full_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse', 'mape'])
full_model.summary()

Model: "Full_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 full_input (InputLayer)     [(None, 58)]              0         
                                                                 
 Dense_128 (Dense)           (None, 128)               7552      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 Dense_256 (Dense)           (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 Dense_64 (Dense)            (None, 64)                16448     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

In [9]:
history_full_model = full_model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.5, verbose=1)

Epoch 1/100
3332/3332 [==============================] - 20s 6ms/step - loss: 2574256.5000 - mae: 592.8298 - mse: 2574256.5000 - mape: 2691.7375 - val_loss: 6709572.5000 - val_mae: 390.5462 - val_mse: 6709572.5000 - val_mape: 1162.7964
Epoch 2/100
3332/3332 [==============================] - 21s 6ms/step - loss: 3115977.7500 - mae: 437.9362 - mse: 3115977.7500 - mape: 1726.6422 - val_loss: 5522454.0000 - val_mae: 324.5039 - val_mse: 5522454.0000 - val_mape: 932.6782
Epoch 3/100
3332/3332 [==============================] - 19s 6ms/step - loss: 2214738.0000 - mae: 333.9312 - mse: 2214738.0000 - mape: 1028.6436 - val_loss: 3332813.5000 - val_mae: 250.2531 - val_mse: 3332813.5000 - val_mape: 710.2819
Epoch 4/100
3332/3332 [==============================] - 20s 6ms/step - loss: 1705590.1250 - mae: 279.8443 - mse: 1705590.1250 - mape: 774.4926 - val_loss: 1502256.6250 - val_mae: 215.6923 - val_mse: 1502256.6250 - val_mape: 692.7458
Epoch 5/100
3332/3332 [==============================] - 20s

In [7]:
def build_model(units=32, dropout_rate=0.2, l2_reg=0.01, patience=10):
    input_shape = X_train_scaled.shape[1]
    
    inputs = keras.Input(shape=(input_shape,))
    
    x = layers.Dense(units, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(inputs)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Dense(units * 2, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Dense(units * 4, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout_rate)(x)
    
    outputs = layers.Dense(1)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse', 'mape'])
    
    return model

In [ ]:
class SavingKerasRegressor(KerasRegressor):
    def fit(self, X, y, **kwargs):
        # Extract current params
        current_params = self.get_params()
        
        # Construct unique filename from current parameters
        filename = "spec_photo_v4.0_units{}_do{}_l2{}_bs{}_ep{}_pat{}.h5".format(
            current_params.get('units'),
            current_params.get('dropout_rate'),
            current_params.get('l2_reg'),
            current_params.get('batch_size'),
            current_params.get('epochs'),
            current_params.get('patience'),
            current_params.get('validation_split')
        )
        os.makedirs(f"history/v4.0", exist_ok=True)
        model_path = os.path.join("models", f"model_{filename}.h5")
        history_path = os.path.join("history/v4.0", f"history_{filename}.json")

        # Callbacks
        checkpoint_cb = ModelCheckpoint(
            model_path,
            save_best_only=True,
            save_format='h5',
            monitor='val_loss',
            mode='min',
            verbose=0
        )
        early_cb = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Add callbacks and validation_split
        kwargs['callbacks'] = [checkpoint_cb, early_cb]
        kwargs['validation_split'] = current_params.get('validation_split')

        history = super().fit(X, y, **kwargs)
        
        # Save history to JSON
        with open(history_path, 'w') as f:
            json.dump(history.history, f)

        return history

In [9]:
regressor = SavingKerasRegressor(build_fn=build_model, verbose=0)

param_grid = {
    'units': [32, 64],
    'dropout_rate': [0.2, 0.3],
    'l2_reg': [0.001, 0.01],
    'patience': [20, 30],
    'batch_size': [16, 32, 64],
    'epochs': [200],
    'validation_split': [0.5, 0.6, 0.7]
}

C:\Users\Sahil\AppData\Local\Temp\ipykernel_31424\3959282293.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  regressor = SavingKerasRegressor(build_fn=build_model, verbose=0)


In [10]:
grid = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=2, scoring='neg_mean_squared_error', verbose=2)
grid_result = grid.fit(X_train_scaled, y_train)

Fitting 2 folds for each of 144 candidates, totalling 288 fits
[CV] END batch_size=16, dropout_rate=0.2, epochs=200, l2_reg=0.001, patience=20, units=32, validation_split=0.5; total time= 8.3min
[CV] END batch_size=16, dropout_rate=0.2, epochs=200, l2_reg=0.001, patience=20, units=32, validation_split=0.5; total time= 8.9min
[CV] END batch_size=16, dropout_rate=0.2, epochs=200, l2_reg=0.001, patience=20, units=32, validation_split=0.6; total time= 6.6min
[CV] END batch_size=16, dropout_rate=0.2, epochs=200, l2_reg=0.001, patience=20, units=32, validation_split=0.6; total time= 6.1min
[CV] END batch_size=16, dropout_rate=0.2, epochs=200, l2_reg=0.001, patience=20, units=32, validation_split=0.7; total time= 3.1min
[CV] END batch_size=16, dropout_rate=0.2, epochs=200, l2_reg=0.001, patience=20, units=32, validation_split=0.7; total time= 3.2min
[CV] END batch_size=16, dropout_rate=0.2, epochs=200, l2_reg=0.001, patience=20, units=64, validation_split=0.5; total time= 6.5min
[CV] END batc

In [11]:
print("Best Score: ", grid_result.best_score_)
print("Best Params: ", grid_result.best_params_)

Best Score:  -549342.2775366982
Best Params:  {'batch_size': 16, 'dropout_rate': 0.2, 'epochs': 200, 'l2_reg': 0.001, 'patience': 20, 'units': 32, 'validation_split': 0.6}
